In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torch import cuda
from torch import device
from torchvision.models import alexnet
from torch import nn
!pip install torch-summary
from torchsummary import summary
from torchvision import transforms
!pip install imutils
from imutils import paths
import cv2
import torch

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25834 sha256=55de5b0c60868f88decf4737377ed2032db13b6cbbfd25dc038f6b2a01e053c5
  Stored in directory: /root/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils
/kaggle/input/bikevshorse/Horses/horse1  (24).jpg
/kaggle/input/bikevshorse/Horses/horse68.jpg
/kaggle/input/bikevshorse/Horses/horse75.jpg
/kaggle/input/bikevshorse/Horses/horse57.jpg
/kaggle/input/bikevshorse/Horses/horse36.jpg
/kaggle/input/bikevshorse/Horses/horse67.jpg
/kaggle/input/bikevshorse/Horses/horse12.jpg
/kaggle/input/bikevshorse/Horses/horse59.jpg
/kaggle/input/bikevshorse/Horses/horse38.jpg
/kaggle/input/bikevshorse/Horses/horse33.jpg
/kaggle/input/bikevshorse/Horses/horse81.jpg
/kaggle/input/bikevshorse/Horses/horse29.jpg
/kaggle/input/bikevshorse/Horses/h1.jpg
/kaggle/input/bikevshorse/Horses/horse32.jpg
/kaggle/input/bikevshorse/Horses/horse65.jp

In [2]:
device = device('cuda:0' if cuda.is_available() else 'cpu')

In [3]:
alexnet = alexnet(weights='AlexNet_Weights.DEFAULT')
print(alexnet)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 128MB/s]  


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
class AlexFeatureExtractor(nn.Module):

  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
    # Extract Feature Layers
    self.features = model.features
    # Extract Average Pooling Layer
    self.pooling = model.avgpool
    # Convert the image into one-dimensional vector
    self.flatten = nn.Flatten()
  
  def forward(self, x):
    # Take image x and return a feature vector
    out = self.features(x)
    out = self.pooling(out)
    out = self.flatten(out)
    return out 

feature_extractor = AlexFeatureExtractor(alexnet)
feature_extractor = feature_extractor.to(device)

In [5]:
print(summary(feature_extractor, input_size=(3, 224, 224)))

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─AdaptiveAvgPool2d: 1-2                 --
├─Flatten: 1-3                           --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0
Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └

In [6]:
transform = transforms.Compose([ transforms.ToPILImage(), transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]), ])

In [7]:
print(transform)

Compose(
    ToPILImage()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [8]:
#Uncomment to use the fruits and vegetables dataset instead -------------------------------------------
# image_paths = list(paths.list_images('/kaggle/input/fruit-and-vegetable-image-recognition'))

#Bike vs Horse dataset --------------------------------------------------------------------------------------------
image_paths = list(paths.list_images('/kaggle/input/bikevshorse'))

In [9]:
def loadDataset(datasetPath):
    imageList = []
    
    directories = [f for f in os.listdir(datasetPath) if os.path.isdir(os.path.join(datasetPath, f))]
    
    labelList = []
    label = -1

    for directory in directories:
        label = str(directory)
        # path = resized+'/'+directory
        path = os.path.join(datasetPath, directory)

        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        
        for file in files:
            imagePath = path+'/'+file
            imageList.append(imagePath)
            labelList.append(label)
    
    return (imageList,labelList)

In [10]:
# datasetPath = '/kaggle/input/fruit-and-vegetable-image-recognition/train'
datasetPath = '/kaggle/input/bikevshorse'

directories = [f for f in os.listdir(datasetPath) if os.path.isdir(os.path.join(datasetPath, f))]
print(directories)

['Horses', 'Bikes']


In [11]:
imageList,labelList = loadDataset(datasetPath)

In [12]:
def getFeatures(imageList):
    features_array = []
    for i in range(len(imageList)):
#         if i == 1165:
#             continue
        img = cv2.imread(imageList[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = transform(img)
        img = img.reshape(1, 3, 224, 224)
        img = img.to(device)
        with torch.no_grad():
            img_features = feature_extractor(img)
        img_features = img_features.cpu().detach().numpy().reshape(-1)
        features_array.append(img_features)
    return features_array


In [26]:
features_array = getFeatures(imageList)

In [27]:
print(len(features_array))

X_train = np.array(features_array)
copyLabelList = labelList
# del copyLabelList[1165]
Y_train = copyLabelList

len(Y_train)

179


179

In [16]:
################## Uncomment while using the fruits and vegetables dataset as a validation/test dataset exists
#Comment the next section in this case

# validationDatasetPath = '/kaggle/input/fruit-and-vegetable-image-recognition/test'

# valImageList, valLabelList = loadDataset(validationDatasetPath)

# valFeatures = getFeatures(valImageList)
# X_val = np.array(valFeatures)
# Y_val = valLabelList

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.5, random_state=42)

X_train.shape

(89, 9216)

In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_val)

array([[-0.26761597, -0.23960438, -0.18389075, ..., -0.22801507,
        -0.18442176, -0.3877792 ],
       [-0.26761597, -0.23960438, -0.18389075, ..., -0.22801507,
         6.9491043 ,  1.1708547 ],
       [-0.26761597, -0.23960438, -0.18389075, ...,  3.2394228 ,
         4.260985  ,  1.9153723 ],
       ...,
       [-0.26761597, -0.23960438, -0.18389075, ...,  2.9329212 ,
        13.388323  ,  4.5416703 ],
       [-0.26761597, -0.23960438, -0.18389075, ..., -0.22801507,
        -0.18442176, -0.3877792 ],
       [-0.26761597, -0.23960438, -0.18389075, ..., -0.22801507,
        -0.18442176, -0.3877792 ]], dtype=float32)

In [30]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=100000).fit(X_train, Y_train)
y_pred = clf.predict(X_val)
clf.score(X_val,Y_val)

0.9888888888888889

In [31]:
from sklearn.svm import SVC

clf = SVC(gamma='auto',kernel='linear')
clf.fit(X_train, Y_train)
clf.score(X_val,Y_val)

0.9888888888888889

In [32]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, random_state=42)
clf.fit(X_train, Y_train)
clf.score(X_val,Y_val)

1.0

In [33]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42)
clf.fit(X_train, Y_train)
clf.score(X_val,Y_val)

1.0

In [34]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=50, random_state=42)
clf.fit(X_train, Y_train)
clf.score(X_val,Y_val)

1.0

In [35]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42,n_estimators=500)
clf.fit(X_train, Y_train)
clf.score(X_val,Y_val)

1.0

In [36]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, Y_train)
neigh.score(X_val,Y_val)

0.9888888888888889